# Gauss Quadrature-Based Green’s Function Evaluation: A Python Implementation

Used packages for the code:

In [ ]:
# --- Standard Library Imports ---
import math
import cmath
import time
import collections

# --- Third-Party Library Imports ---
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.integrate import quad
from scipy.special import kv as besselk, roots_laguerre

**Some additions:**

* This function computes the integral of a complex-valued function over [a, b] by integrating its real and imaginary parts separately before combining the results:

In [ ]:
def complex_quad(func, a, b, **kwargs):
    real_integral, _ = quad(lambda x: np.real(func(x)), a, b, **kwargs)
    imag_integral, _ = quad(lambda x: np.imag(func(x)), a, b, **kwargs)
    return real_integral + 1j * imag_integral

 * this function replaces all 'inf' values in a NumPy array with the mean of the other finite values.

In [ ]:
def replace_inf_with_mean(arr):
    """
    Replaces all 'inf' values in a NumPy array with the mean of the other finite values.

    Parameters:
    -----------
    arr : np.ndarray
        Input array that may contain 'inf' values.

    Returns:
    --------
    np.ndarray
        Array with 'inf' values replaced by the mean of the other finite values.
    """
    finite_values = arr[np.isfinite(arr)]  # Extract all finite values
    mean_value = np.mean(finite_values) if finite_values.size > 0 else 0  # Compute mean, or use 0 if empty
    arr[np.isinf(arr)] = mean_value  # Replace 'inf' values with the mean
    return arr

## The Green's function

The Green’s function is given by
$$
G(\mathbf{x}, \mathbf{y}) = G_{\text{full}}(\mathbf{x} - \mathbf{y}) + G_{\text{refl}}(\mathbf{x} - \mathbf{Ry}) + G_{\text{imp}}(\mathbf{x} - \mathbf{Ry})
$$
with
$$
G_{\text{full}}(\mathbf{z}) := g_{\nu}(\|\mathbf{z}\|), \quad G_{\text{refl}}(\mathbf{z}) := \sigma_{\nu}(\|\mathbf{z}\|, z_d).
$$

The definition of $G_{\text{imp}}$ requires some preparation. For $\mathbf{z} = (z_j)_{j=1}^{d} \in \mathbb{R}^d$ and $\mathbf{z}' = (z_j)_{j=1}^{d-1}$, we introduce
\begin{align*}
\, & \hat{a} := \hat{a}(\eta, \rho, \zeta) := \eta + \rho + \beta \zeta\\
\, & \hat{\chi} := \hat{\chi}(\eta, \rho, \zeta) := \hat{a}^2 - (1 - \beta^2)(\rho^2 - \zeta^2)\\
\, & \hat{\mu} := \hat{\mu}(\eta, \rho, \zeta) := \frac{\hat{a} \sqrt{\hat{\chi}} + \beta (\rho^2 - \zeta^2)}{\beta \hat{a} + \sqrt{\hat{\chi}}}\\
\, & \hat{t} := \hat{t}(\eta, \rho, \zeta) := \frac{\hat{a}^2 - (\rho^2 - \zeta^2)}{\beta \hat{a} + \sqrt{\hat{\chi}}}\\
\, & q_{\nu} := q_{\nu}(\eta, \rho, \zeta) := \frac{d}{d\eta} \left(\frac{ e^{\hat{\mu}} K_{\nu+1/2}(\hat{\mu})}{ \left(\hat{t} + \beta \hat{\mu} \right) \hat{\mu} \, ^{\nu - 1/2} }\right) 
\end{align*}

which enter the definition of $G_{\text{imp}}$ via
\begin{equation}
G_{\text{imp}}(\mathbf{z}) = - \frac{\beta}{\pi} \left( \frac{s^2}{2\pi} \right)^{\nu + 1/2} e^{-sr} \psi_{\nu}(sr, s z_d) 
\end{equation}
for
\begin{equation}
\psi_{\nu}(\rho, \zeta) := \int_0^{\infty} e^{-\eta} q_{\nu}(\eta, \rho, \zeta) \, d\eta
\end{equation}

After algebraic manipulation, the explicit form of the integrand is obtained as follows:
\begin{equation}
q_{\nu} = \frac{1}{\left( \hat{t} + \beta \hat{\mu} \right)^2} 
\left( -\frac{\rho^2 - \zeta^2}{\hat{t} + \beta \hat{\mu}} \,  \frac{e^{\hat{\mu}} K_{\nu + 1/2}(\hat{\mu}) }{\hat{\mu}^{\nu + 1/2} }
+ \hat{t}\, e^{\hat{\mu}}\, \frac{ K_{\nu + 1/2}(\hat{\mu}) - K_{\nu + 3/2}(\hat{\mu})}{ \hat{\mu}^{\nu - 1/2}} \right).
\end{equation}

 With $\nu$ be the he dependence on the spatial dimension $d$
\begin{equation}
\nu := (d - 3)/2.
\end{equation}

Constraints for parameters $\beta$ and $\nu$:

In [ ]:
beta = 0.1
nu = 1

Since we used the constant $ \nu = 1 $ throughout the entire work, the modified Bessel function of the second kind, $ K_\omega(x) $, can be explicitly expressed for half-integer values of $ \omega $.

The modified Bessel function $ K_{\frac{3}{2}}(x) $ is given by  

$$
K_{\frac{3}{2}}(x) = \sqrt{\frac{\pi}{2x}} e^{-x} \left( 1 + \frac{1}{2x} \right)
$$

and the modified Bessel function $ K_{\frac{5}{2}}(x) $ is  

$$
K_{\frac{5}{2}}(x) = \sqrt{\frac{\pi}{2x}} e^{-x} \left( 1 + \frac{3}{2x} + \frac{3}{(2x)^2} \right)
$$

In [ ]:
def Green_fct( s, r, zd):
    """
    Evaluates the Green’s function using numerical quadrature methods

    Parameters:
    -----------
    s : float
        Scaling parameter.
    r : float
        Radial coordinate.
    zd : float
        Depth coordinate.

    Returns:
    --------
    G_real : float
        Real part of the computed Green's function.
    G_imag : float
        Imaginary part of the computed Green's function.
    """
    # Compute rho and zeta
    rho = s * r
    zeta = s * zd

    a_hat = lambda eta: eta + rho + beta * zeta
    hi_hat = lambda eta: a_hat(eta) ** 2 - (1 - beta ** 2) * (rho ** 2 - zeta ** 2)

    # Clips values to prevent numerical overflows in mu_hat
    def mu_hat(eta):
        mu_value = (a_hat(eta) * np.sqrt(hi_hat(eta)) + beta * (rho ** 2 - zeta ** 2)) / (
                    beta * a_hat(eta) + np.sqrt(hi_hat(eta)))
        return np.clip(mu_value, -700, 700)
        mu_value_clipped = np.clip(mu_value, -700, 700)

        return mu_value_clipped

    t_hat = lambda eta: (a_hat(eta) ** 2 - (rho ** 2 - zeta ** 2)) / (beta * a_hat(eta) + np.sqrt(hi_hat(eta)))

    # Compute the modified Bessel functions of the second kind
    K1 = lambda eta: besselk(nu + 1 / 2, mu_hat(eta))
    K2 = lambda eta: besselk(nu + 3 / 2, mu_hat(eta))

    # Direct computation for ν = 1 modified Bessel function of the second kind
    # K1 = lambda eta: np.sqrt(np.pi / (2 * mu_hat(eta))) * np.exp(-mu_hat(eta)) * (1 + 1 / (2 * mu_hat(eta)))
    # K2 = lambda eta: np.sqrt(np.pi / (2 * mu_hat(eta))) * np.exp(-mu_hat(eta)) * (1 + 3 / (2 * mu_hat(eta)) + 3 / (4 * mu_hat(eta)**2))

    q_nu = lambda eta: (
            1 / (t_hat(eta) + beta * mu_hat(eta)) ** 2 *
            (
                    -1 * (rho ** 2 - zeta ** 2) * np.exp(mu_hat(eta)) * K1(eta) /
                    (mu_hat(eta) ** (nu + 1 / 2) * (t_hat(eta) + beta * mu_hat(eta))) +

                    t_hat(eta) * np.exp(mu_hat(eta)) *
                    (K1(eta) - K2(eta)) /
                    (mu_hat(eta) ** (nu - 1 / 2))
            )
    )

    # Compute the Green's function
    exp_q_nu = lambda eta: np.exp(-eta) * q_nu(eta)

    psi_nu = complex_quad(exp_q_nu, 0, np.inf)

    G_imp = (- beta / np.pi) * (s ** 2 / (2 * np.pi)) ** (nu + 1 / 2) * cmath.exp(- rho) * psi_nu

    return G_imp.real, G_imp.imag


Parameters $s$, $r$, $z_d$ for Green's function

In [ ]:

s_values = [1/2 + 1/2j, 1/2 + 10j, 1 + 30j]

r_min=1
r_max=3
r_axis = np.linspace(r_min, r_max, 300)

zd=0

Plot settings:

In [ ]:
color1 = "mediumblue"
color2 = "blueviolet"
color3 = "mediumvioletred"
color4 = "black"

fig, axs = plt.subplots(len(s_values), 1, figsize=(8, 12))

for i, s in enumerate(s_values):
    G_values_real, G_values_imag = zip(*[Green_fct(s, r, zd) for r in r_axis])
    axs[i].plot(r_axis, G_values_real, label="Re(G_imp)", color=color1)
    axs[i].plot(r_axis, G_values_imag, label="Im(G_imp)", color=color3)
    axs[i].set_title(f"$s = {s.real:.3f} + i{s.imag:.3f}$")
    axs[i].legend()
    axs[i].set_xlabel("$r$")
    axs[i].grid(True, linestyle='--', alpha=0.1)

fig.suptitle(r"Variation of the Impedance Green's Function $G_{\text{imp}}(\mathbf{z})$ with Complex Parameter $s$", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.97])  # Damit der Haupttitel nicht mit Subplots kollidiert
plt.show()

## Fast evaluation of $\psi_{\nu}$

To evaluate the integral
$$
\psi_{\nu}(\rho, \zeta) = \int_{0}^{\infty} \mathrm{e}^{-\eta}\, q_{\nu}(\eta, \rho, \zeta) \, \mathrm{d}\eta,
$$
we need to consider the value of $|\rho|$ and decide on the appropriate strategy based on its magnitude. 

If $|\rho|$ is **not small**, standard Gauss-Laguerre quadrature converges exponentially and can be directly applied to approximate the integral efficiently.

If $|\rho|$ is **small**, a geometric subdivision of the integration domain is necessary, and we apply Gauss-Legendre quadrature for the initial segments of the domain while using Gauss-Laguerre quadrature for the remaining part. 

For the next visualization of the function $ \psi_{\nu}(\rho, \zeta) $, the arguments of the function are defined as
$$
\rho = 2^{l}, \quad \zeta = \frac{\rho}{2}
$$
where $ l \in \mathbb{R}_{-} $ for the part of the plot where $ |\rho| $ is small, and $ l \in \mathbb{R}_{+} $ for the part of the plot where $ |\rho| $ is large.

In [ ]:
def psi_nu(l, index):

    """
    Computes the function psi_nu using a quadrature-based approach for different scaling regimes.

    Parameters:
    -----------
    l : int
        Scaling parameter controlling the interval transformation.
    index : int
        Defines the regime:
        - index = 1: Large rho (not small)
        - index ≠ 1: Small rho

    Returns:
    --------
    psi_nu_real : float
        The real part of the computed integral.
    x_0_rho : numpy.ndarray
        Discretized values in the range [0, rho].
    x_rho_1 : numpy.ndarray
        Discretized values in the range [rho, 1].
    x_1_inf : numpy.ndarray
        Discretized values in the range [1, 3].
        
    exp_q_nu_0_rho : numpy.ndarray
        Evaluations of exp_q_nu over [0, rho].
    exp_q_nu_rho_1 : numpy.ndarray
        Evaluations of exp_q_nu over [rho, 1].
    exp_q_nu_1_inf : numpy.ndarray
        Evaluations of exp_q_nu over [1, 3].
    """

    # Compute rho and zeta based on index
    if index == 1: #not small ρ
        rho = 2.0 ** l
        zeta = rho / 2.0

    else: # small ρ
        rho = 2.0 ** (-l)
        zeta = rho / 2.0

    a_hat = lambda eta: eta + rho + beta * zeta
    hi_hat = lambda eta: a_hat(eta) ** 2 - (1 - beta ** 2) * (rho ** 2 - zeta ** 2)

    # Value limitation to prevent overflows
    def mu_hat(eta):
        mu_value = (a_hat(eta) * np.sqrt(hi_hat(eta)) + beta * (rho ** 2 - zeta ** 2)) / (
                    beta * a_hat(eta) + np.sqrt(hi_hat(eta)))
        return np.clip(mu_value, -700, 700)
        mu_value_clipped = np.clip(mu_value, -700, 700)

        return mu_value_clipped

    t_hat = lambda eta: (a_hat(eta) ** 2 - (rho ** 2 - zeta ** 2)) / (beta * a_hat(eta) + np.sqrt(hi_hat(eta)))

    #  Compute the modified Bessel functions of the second kind
    K1 = lambda eta: besselk(nu + 1 / 2, mu_hat(eta))
    K2 = lambda eta: besselk(nu + 3 / 2, mu_hat(eta))

    # Direct computation for ν = 1 modified Bessel function of the second kind
    # K1 = lambda eta: np.sqrt(np.pi / (2 * mu_hat(eta))) * np.exp(-mu_hat(eta)) * (1 + 1 / (2 * mu_hat(eta)))
    # K2 = lambda eta: np.sqrt(np.pi / (2 * mu_hat(eta))) * np.exp(-mu_hat(eta)) * (1 + 3 / (2 * mu_hat(eta)) + 3 / (4 * mu_hat(eta)**2))

    q_nu = lambda eta: (
            1 / (t_hat(eta) + beta * mu_hat(eta)) ** 2 *
            (
                    -1 * (rho ** 2 - zeta ** 2) * np.exp(mu_hat(eta)) * K1(eta) /
                    (mu_hat(eta) ** (nu + 1 / 2) * (t_hat(eta) + beta * mu_hat(eta))) +

                    t_hat(eta) * np.exp(mu_hat(eta)) *
                    (K1(eta) - K2(eta)) /
                    (mu_hat(eta) ** (nu - 1 / 2))
            )
    )
    exp_q_nu = lambda eta: np.exp(-eta) * q_nu(eta)
    psi_nu = complex_quad(exp_q_nu, 0, np.inf)

    
    # Define discretized domains for function evaluation
    x_0_rho = np.linspace(0, rho, 100)
    x_rho_1 = np.linspace(rho, 1, 100)
    x_1_inf = np.linspace( 1, 3, 100)

    # Evaluate exp_q_nu over defined intervals
    exp_q_nu_0_rho = exp_q_nu(x_0_rho)
    exp_q_nu_rho_1 = exp_q_nu(x_rho_1)
    xp_q_nu_1_inf = exp_q_nu(x_1_inf)


    return psi_nu.real, x_0_rho, x_rho_1, x_1_inf, exp_q_nu_0_rho, exp_q_nu_rho_1, xp_q_nu_1_inf

Plot settings:

In [ ]:
l_max = 10
l_values = np.arange (0.1,l_max, 0.1)

# Computation of psi_nu for small \rho
psi_nu_small = [psi_nu(l, 0)[0] for l in l_values]
# Computation of psi_nu for \rho NOT small
psi_nu_NOT_small = [psi_nu(l, 1)[0] for l in l_values]

fig, axs = plt.subplots(2, 1, figsize=(8, 12))

# Plot für psi_nu_small
axs[0].plot(l_values, psi_nu_small, label=r'$\psi_\nu$', color="blue")
axs[0].set_xlabel(r'$l$', fontsize=16)
axs[0].set_ylabel(r'$\psi_\nu$', fontsize=14)
axs[0].set_title(r'Small $|\rho|$: $\rho=2^{-l}$')
for l in range(1, l_max):
    axs[0].scatter(l, psi_nu(l, 0)[0], color="mediumblue", edgecolors="black", marker="o", s=40, zorder=3)

axs[0].grid(True, linestyle='--', alpha=0.5)
axs[0].legend()

# Plot für psi_nu_NOT_small
axs[1].plot(l_values, psi_nu_NOT_small, label=r'$\psi_\nu$', color="blueviolet")
axs[1].set_xlabel(r'$l$', fontsize=16)
axs[1].set_ylabel(r'$\psi_\nu$', fontsize=14)
axs[1].set_title(r'NOT small $|\rho|$: $\rho=2^{l}$')
for l in range(1, l_max):
    axs[1].scatter(l, psi_nu(l, 1)[0], color="rebeccapurple", edgecolors="black", marker="o", s=40, zorder=3)

axs[1].grid(True, linestyle='--', alpha=0.5)
axs[1].legend()

fig.suptitle(r'Numerical Behavior of $\psi_\nu(\rho, \zeta)$', fontsize=16)
plt.tight_layout()
plt.show()

# Layout anpassen
plt.tight_layout()
plt.show()

From this point forward, we assume that $ \rho $ follows the form  

$$
\rho = 2^{-l}, \quad \zeta = \frac{\rho}{2}
$$

where $ l = 1, 2, 3, 4, \dots $.  

This choice of $ \rho $ and $ \zeta $ allows for a systematic and structured examination of how changes in these parameters affect the integral evaluation. By expressing $ \rho $ as a negative power of two, we introduce a logically ordered sequence that facilitates a step-by-step analysis of the function’s behavior across different scales.

#### Integral Approximation Cases

**Case 1:** If $|\rho| \geq 1$, the integral  
$$
\int_{0}^\infty \mathrm{e}^{-\eta} q_\nu (\eta, \rho, \zeta) \, \mathrm{d}\eta
$$  
is approximated using Gauss-Laguerre quadrature (see, e.g., [Bouillot 2023]), which guarantees exponential convergence.

**Case 2:** Let $0 < |\rho| < 1$. For this case, the integral is split as follows:  
\begin{equation}
\psi_{\nu}(\rho, \zeta) = \psi_{\nu}^{\text{GLeg}}(\rho, \zeta) + \psi_{\nu}^{\text{hp}}(\rho, \zeta) + \psi_{\nu}^{\text{GLag}}(\rho, \zeta),
\end{equation}
where each term corresponds to a specific subinterval of the integration domain:
\begin{equation*}
\begin{split}
\psi_\nu^{\text{GLeg}}(\rho, \zeta) &:= \int_0^{|\rho|} \mathrm{e}^{-\eta} q_\nu (\eta, \rho, \zeta) \, \mathrm{d}\eta, \\[11pt]
\psi_\nu^{\text{hp}}(\rho, \zeta) &:= \int_{|\rho|}^1 \mathrm{e}^{-\eta} q_\nu (\eta, \rho, \zeta) \, \mathrm{d}\eta, \\[11pt]
\psi_\nu^{\text{GLag}}(\rho, \zeta) &:= \int_1^\infty \mathrm{e}^{-\eta} q_\nu (\eta, \rho, \zeta) \, \mathrm{d}\eta.
\end{split}
\end{equation*}

In the next three subsections, we focus primarily on the second case.


A plot illustrating the behavior of the function $ \psi_\nu(\rho, \zeta) $ across the three distinct integration subintervals for different values of $ l $.

In [ ]:
# Values of l to be plotted
l_values = [1, 4, 7, 10] 

In [ ]:
fig, axs = plt.subplots(len(l_values), 1, figsize=(8, 12))

# If axs is not an iterable object
if len(l_values) == 1:
    axs = [axs]

for i, l in enumerate(l_values):
    _, x1, x2, x3, y1, y2, y3 = psi_nu(l,0)

    color1 = "mediumblue"
    color2 = "blueviolet"
    color3 = "mediumvioletred"
    color4 = "black"

    axs[i].plot(x1, y1, color=color1, linewidth=3, label=fr"$\left[0, 2^{{{l}}}\right]$")
    axs[i].plot(x2, y2, color=color2, linewidth=3, label=fr"$\left[2^{{{l}}}, 1\right]$")
    axs[i].plot(x3, y3, color=color3, linewidth=3, label=fr"$\left[1, \infty\right]$")

    axs[i].scatter(x1[0], y1[0], color=color4, edgecolors="black", marker="o", s=40, zorder=3)
    axs[i].scatter(x2[0], y2[0], color=color4, edgecolors="black", marker="o", s=40, zorder=3)
    axs[i].scatter(x3[0], y3[0], color=color4, edgecolors="black", marker="o", s=40, zorder=3)

    axs[i].set_ylabel(r'Integrand', fontsize=12)
    axs[i].set_title(f"l = {l}", fontsize=13)  
    axs[i].legend(fontsize=10)  
    axs[i].grid(True, linestyle='--', alpha=0.5) 
    
axs[-1].set_xlabel(r"$\eta$", fontsize=14)
fig.suptitle(r'Integrand $e^{-\eta} q_{\nu}(\eta,\rho, \zeta)$ for different $l$', fontsize=16)

plt.tight_layout(rect=[0, 0, 1, 0.97]) 
plt.show()


## Quadrature Methods: Numerical Integration Codes

### Gauss-Legendre quadrature

The following code implements Gauss-Legendre quadrature by computing the optimal quadrature nodes and weights to approximate the integral of a given function $f$ over an arbitrary interval $[a, b]$ using $n$ nodes, ensuring high accuracy for smooth functions.

In [ ]:
def gauss_legendre_quadrature_ab(f, a, b, n):
    """
    Computes the Gauss-Legendre quadrature approximation of the integral of f over [a, b].

    Parameters:
    -----------
    f : function
        The function to be integrated.
    a : float
        The lower bound of the integration interval.
    b : float
        The upper bound of the integration interval.
    n : int
        The number of quadrature nodesThe following code calculates the quadrature nodes (support points) and weights required for Gauss-Legendre quadrature. It then applies this numerical integration technique to approximate the integral of a given function $f$ over an arbitrary interval $[a, b]$ using $n$ quadrature nodes. The Gauss-Legendre quadrature method is particularly effective for smooth functions, as it achieves high accuracy with relatively few nodes by leveraging the optimal placement of integration points.
.

    Returns:
    --------
    float
        Approximate integral of f over [a, b] using Gauss-Legendre quadrature.
    """
    # Compute Gauss-Legendre nodes (x) and weights (w)
    nodes, weights = np.polynomial.legendre.leggauss(n)
    
    # Transform nodes to the given interval [a, b]
    transformed_nodes = (b - a) / 2 * nodes + (b + a) / 2
    transformed_weights = (b - a) / 2 * weights

    # Compute the quadrature sum
    integral_approximation = np.sum(transformed_weights * f(transformed_nodes))

    return integral_approximation

### Gauss-Legendre quadrature on a Geometrically Graded Mesh ($hp$-Gauss)

This part is evaluated using a geometrically graded mesh with a fixed grading parameter $\sigma \in (0,1)$ and a slope parameter $\mu$.

In [ ]:
sigma = 1/2
mu = 2

The mesh points are defined as  

$$
\xi_i := \sigma^i, \quad i \in \{0, 1, \dots, n\},
$$  

where $n$ is the largest integer such that $\xi_n > 0$. Finally, we set $\xi_{n+1} = 0$ and define the subintervals as  

$$
\tau_{n+1-i} := \left[\xi_{i+1}, \xi_i\right], \quad 0 \leq i \leq n.
$$  

The integral over each subinterval $\tau_i$ is approximated using Gauss-Legendre quadrature. The quadrature is appropriately scaled to each interval $\tau_i$ through an affine pullback to the reference interval $\left[-1, 1\right]$. The number of Gauss-Legendre quadrature points is determined adaptively based on the length of each interval $\tau_i$.  

For one-dimensional integration, the $hp$-Gauss quadrature of any continuous, integrable function $f$ is defined, where $p_j$ represents the polynomial degree on the panel $\tau_j$, determined by  

$$
p_j = \max\{2, \lfloor j\mu \rfloor + 1\}.
$$

The following code implements an adaptive $hp$-Gauss-Legendre quadrature scheme by computing optimal quadrature nodes and weights over a geometrically graded mesh. It approximates the integral of a given function $f$ over an arbitrary interval $[0, 1]$, where the number of quadrature nodes is adaptively determined based on the interval length and the polynomial degree $p_j$, ensuring high accuracy even for singular or rapidly varying functions.

**Note:** Before using this function, a change of variables should be applied to transform the integral to the standard interval $[0, 1]$


In [ ]:
def hp_gauss_legendre_quadrature(f, sigma, mu, n):
    """
    Computes the hp-adaptive Gauss-Legendre quadrature for the function f over a geometrically 
    graded mesh with a grading parameter sigma and a slope parameter mu.

    Parameters:
    -----------
    f : function
        The function to be integrated over the interval [0, 1].
    sigma : float
        The grading parameter, with 0 < sigma < 1.
    mu : float
        The slope parameter controlling the polynomial degree distribution.
    n : int
        The number of subintervals in the graded mesh.

    Returns:
    --------
    I : float
        The computed integral approximation.
    N : int
        The total number of quadrature points used.
    """
    integral = 0.0
    total_nodes = 0

    for j in range(n + 1):
        # Compute polynomial degree p_j for the current subinterval
        p_j = max(2, int((n - j) * mu) + 1)

        # Compute Gauss-Legendre quadrature for the subinterval [sigma^(j+1), sigma^j]
        integral += gauss_legendre_quadrature_ab(f, sigma ** (j + 1), sigma ** j, p_j)

        # Accumulate the total number of quadrature points
        total_nodes += p_j

    return integral, total_nodes

### Gauss-Laguerre quadrature

The following code implements Gauss-Laguerre quadrature by computing optimal quadrature nodes and weights to approximate the integral of a given function $f$ over a semi-infinite interval using $n$ nodes. This method is particularly well-suited for integrals of the form  

$$
\int_{0}^{\infty} e^{-x} f(x) \,dx,
$$  

where the weight function $e^{-x}$ is inherently incorporated into the quadrature formula.  The general Gauss-Laguerre quadrature formula given by
\begin{equation}\label{eq:GLag}
    \int_0^{\infty} w_{\alpha}(x)f(x)dx = \sum_{i=1}^{n} \lambda_{ni}^{\alpha} f(x_{ni}^{\alpha}) + R_n(f),
\end{equation}
o adapt the quadrature formula for practical use, we introduce a "truncated" version, defined as:
\begin{equation}\label{eq:thetaGLag}
    \int_0^{\infty} w_{\alpha}(x)f(x)dx = \sum_{0 < x_{ni}^{\alpha} \leq 4\theta n} \lambda_{ni}^{\alpha} f(x_{ni}^{\alpha}) + R_n^{\theta}(f),
\end{equation}
where $0 < \theta < 1$ is an arbitrarily chosen truncation parameter.

**Note:** If the given integral is originally defined over a finite domain, an appropriate change of variables should be applied to transform it to the semi-infinite interval $[0, \infty)$.  



In [4]:
#chose of truncation parameter
theta = 1/4

In [ ]:
def gauss_laguerre_quadrature(f, n, theta):
    """
    Computes the Gauss-Laguerre quadrature approximation of the integral of f over [0, ∞).

    This method is particularly useful for integrals of the form:∫₀^∞ e^(-x) f(x) dx
    where the weight function w(x) = e^(-x) is already incorporated in the quadrature.

    Parameters:
    -----------
    f : function
        The function to be integrated.
    n : int
        The number of quadrature nodes.
    theta: float
        A constant in the interval (0,1) that determines the truncation of the quadrature rule.

    Returns:
    --------
    float
        Approximate integral of f over [0, ∞) using Gauss-Laguerre quadrature.
    """
    if not (0 < theta < 1):
        raise ValueError("Theta must be in the interval (0,1).")

    # Compute Gauss-Laguerre nodes (x) and weights (w)
    nodes, weights = roots_laguerre(n)

    truncation_threshold = 4 * theta * n
    mask = (nodes > 0) & (nodes <= truncation_threshold)
    filtered_nodes = nodes[mask]
    filtered_weights = weights[mask]

    # Compute the quadrature sum
    integral_approximation = np.sum(filtered_weights * f(filtered_nodes))

    return integral_approximation

## Application of Quadrature Methods to the Function $\psi_\nu$

The following codes perform a systematic error analysis for different quadrature methods by computing numerical approximations of $\psi_\nu$ for various values of $n$ and $l$. It evaluates the total number of quadrature nodes, relative errors, and method-specific errors, and visualizes the results using semilogarithmic and linear plots.


In [ ]:
def compute_psi_nu_quadrature(n, l, index):
    """
    Computes the numerical quadrature of the function psi_nu using different quadrature methods.

    Parameters:
    -----------
    n : int
        Number of quadrature nodes.
    l : int
        Scaling parameter controlling the interval transformation.
    index : int
        Determines the computation method:
        - index = 1: Computes only the Gauss-Laguerre quadrature on the entire interval (0,∞) .
        - index ≠ 1: Computes all quadrature methods and compares results.


    Returns:
    --------
    psi_nu : float
        The computed integral using complex quadrature.
    Q_psi_nu : float
        The computed integral using Gauss quadrature.
    N : int
        Total number of quadrature points used for the hp-Gauss part.
    psi_nu_vec : list of float
        Contributions of each quadrature subinterval.
    Q_psi_nu_vec : list of float
        Contributions from Gauss quadrature methods.
    time_result : float
        Stores the computation time required to perform the quadrature.
    """

    # Define rho and zeta
    rho = 2.0 ** (-l)
    zeta = rho / 2

    # Error handling: Ensure |ρ| >= |ζ| and |ρ| <= 1
    if abs(rho) < abs(zeta):
        raise ValueError(f"Constraint violated: |ρ| must be greater than or equal to |ζ|.")
    if abs(rho) >= 1:
        raise ValueError(f"Constraint violated: |ρ| must be less than or equal to 1. Received |ρ| = {abs(rho)}.")

    a_hat = lambda eta: eta + rho + beta * zeta
    hi_hat = lambda eta: a_hat(eta) ** 2 - (1 - beta ** 2) * (rho ** 2 - zeta ** 2)
    mu_hat = lambda eta: (a_hat(eta) * np.sqrt(hi_hat(eta)) + beta * (rho ** 2 - zeta ** 2)) / (
            beta * a_hat(eta) + np.sqrt(hi_hat(eta)))

    t_hat = lambda eta: (a_hat(eta) ** 2 - (rho ** 2 - zeta ** 2)) / (beta * a_hat(eta) + np.sqrt(hi_hat(eta)))

    # ------Computation of q_nu for nu=1 (Special case)-------
    # Direct computation for ν = 1 modified Bessel function of the second kind
    exp_K1 = lambda eta: np.sqrt(np.pi / (2 * mu_hat(eta))) * (1 + 1 / (2 * mu_hat(eta)))
    exp_K2 = lambda eta: np.sqrt(np.pi / (2 * mu_hat(eta))) * (1 + 3 / (2 * mu_hat(eta)) + 3 / (4 * mu_hat(eta)**2))

    # Define the quadrature function q_nu
    q_nu = lambda eta: (
            1 / (t_hat(eta) + beta * mu_hat(eta)) ** 2 *
            (
                    -1 * (rho ** 2 - zeta ** 2) * exp_K1(eta) /
                    (mu_hat(eta) ** (nu + 1 / 2) * (t_hat(eta) + beta * mu_hat(eta))) +

                    t_hat(eta) *
                    (exp_K1(eta) - exp_K2(eta)) /
                    (mu_hat(eta) ** (nu - 1 / 2))
            )
    )

    # Exponential weighting
    exp_q_nu = lambda eta: np.exp(-eta) * q_nu(eta)

    # Gauss-Laguerre Quadrature over the entire interval (0, ∞)
    if index == 1:
        psi_nu_GLag = gauss_laguerre_quadrature(lambda eta: exp_q_nu(eta), n, theta)
        return psi_nu_GLag

    else:
        # ** Gauss-Legendre Quadrature **
        psi_nu_I = complex_quad(exp_q_nu, 0, np.abs(rho))
        start_time = time.time()
        psi_nu_GLeg = gauss_legendre_quadrature_ab(exp_q_nu, 0, np.abs(rho), n)
        end_time = time.time()
        delta_time1 = end_time - start_time

        # ** Gauss-Laguerre Quadrature **
        psi_nu_II = complex_quad(exp_q_nu, 1, np.inf)
        start_time = time.time()
        # Change of variables for Gauss-Laguerre quadrature
        q_nu_cv = lambda eta: q_nu(eta + 1) * 1 / np.exp(1)
        psi_nu_GLag = gauss_laguerre_quadrature(lambda eta: q_nu_cv(eta), n, theta)
        end_time = time.time()
        delta_time2 = end_time - start_time

        # ** hp-Gauss Quadrature **
        # Change of variables to the interval [0,1]
        #psi_nu_III = complex_quad(exp_q_nu, np.abs(rho), 1)
        exp_q_nu_cv = lambda y: (1 - np.abs(rho)) * exp_q_nu(np.abs(rho) * (1 - y) + y)
        psi_nu_III = abs(complex_quad(exp_q_nu_cv, 0, 1))
        start_time = time.time()
        psi_nu_hp_Gauss_GLeg, N = np.abs(hp_gauss_legendre_quadrature(exp_q_nu_cv, sigma, mu, n))
        end_time = time.time()
        delta_time3 = end_time - start_time

        # ** Final Integration Results **
        psi_nu_vec = [psi_nu_I, psi_nu_II, psi_nu_III]
        Q_psi_nu_vec = [psi_nu_GLeg, psi_nu_GLag, psi_nu_hp_Gauss_GLeg]

        psi_nu = psi_nu_I + psi_nu_II + psi_nu_III
        Q_psi_nu = psi_nu_GLeg + psi_nu_GLag + psi_nu_hp_Gauss_GLeg
        time_result = delta_time1 + delta_time2 + delta_time3

        return psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result

This defines the quadrature parameters, where `n_values` represents the number of quadrature points, and `l_values` specifies the maximum exponent of $\rho$ considered for the integral.

In [ ]:
# Define quadrature parameters
n_values = np.arange(2,10)
l_values = np.arange(1,5)

Initialize arrays to store results

In [ ]:
num_iterations = np.zeros((len(l_values), len(n_values)))
relative_error1 = np.zeros((len(l_values), len(n_values)))
relative_error2 = np.zeros((len(l_values), len(n_values)))
relative_error3 = np.zeros((len(l_values), len(n_values)))

Compute errors for each combination of $n$ and $l$

In [ ]:
for i, n in enumerate(n_values):
    for j, l in enumerate(l_values):
        # Compute psi_nu and quadrature approximations
        psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)

        num_iterations[j, i] = N

        relative_error1[j, i] = np.abs(np.abs(psi_nu_vec[0]) - np.abs(Q_psi_nu_vec[0])) / np.abs(psi_nu_vec[0])
        relative_error2[j, i] = np.abs(np.abs(psi_nu_vec[1]) - np.abs(Q_psi_nu_vec[1])) / np.abs(psi_nu_vec[1])
        relative_error3[j, i] = np.abs(np.abs(psi_nu_vec[2]) - np.abs(Q_psi_nu_vec[2])) / np.abs(psi_nu_vec[2])

Plot the relative error values

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16.5, 8.9))
plt.suptitle("Semilogarithmic Plots of Relative Error", fontsize=16)

#-------GLeg-------------
for j, l in enumerate(l_values):
    axs[0].semilogy(n_values_GLeg ** (1/2), relative_error_GLeg[j, :], label=f"l={l}")

axs[0].grid(True, linestyle='--', color='gray', linewidth=0.2)
axs[0].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$')
axs[0].set_title("GLeg")

#-------GLag-------------
for j, l in enumerate(l_values):
    axs[2].semilogy(n_values_GLag ** (1/2), relative_error_GLag[j, :], label=f"l={l}")

axs[2].grid(True, linestyle='--', color='gray', linewidth=0.2)
axs[2].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$')
axs[2].set_title("GLag")

#-------hp-Gauss-------------
for j, l in enumerate(l_values):
    axs[1].semilogy(num_iterations[j, :] ** (1/2), relative_error_hp_Gauss[j, :], label=f"l={l}")

axs[1].grid(True, linestyle='--', color='gray', linewidth=0.2)
axs[1].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$')
axs[1].set_title("hp-GLeg")

#axs[1].set_title("Linear Plot of Relative Error")
plt.subplots_adjust(hspace=0.2, wspace=0.3)
axs[0].set_ylabel("Relative Error")
#axs[2].legend()
#axs[2].legend(loc="center left", bbox_to_anchor=(1, 0.5))

legend = fig.legend(["l=1", "l=2", "l=3", "l=4", "l=5", "l=6", "l=7", "l=8", "l=9", "l=10"], loc="upper center", bbox_to_anchor=(0.5, 0.95), ncol=10,
                        fontsize=9)

# Speichern des Plots mit hoher Qualität
plt.savefig("Err Comp Quadratur for 3 models (trunted rule).png", dpi=600, bbox_inches='tight')


plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Illustrate the individual contribution of each quadrature method to the overall integration accuracy

The `compute_psi_nu_quadrature_individual` function calculates the numerical quadrature of the function `psi_nu` using different quadrature methods based on the provided parameters `n`, `l`, and `index`. Depending on the value of `index`, it performs computations using specific quadrature techniques like Gauss-Legendre, Gauss-Laguerre, or hp-Gauss, and returns a tuple with the results from these methods.

In [ ]:
def compute_psi_nu_quadrature_individual(n, l, index):
    """
    Computes the numerical quadrature of the function psi_nu using different quadrature methods.
    
    Parameters:
    -----------
        n (int): Number of quadrature nodes.
        l (int): Scaling parameter controlling the interval transformation.
        index (int): Determines the computation method.

    Returns:
    -----------
        tuple: Contains results from different quadrature methods.
    """

    # Define rho and zeta
    rho = 2.0 ** (-l)
    zeta = rho / 2

    # Error handling: Ensure |ρ| >= |ζ| and |ρ| <= 1
    if abs(rho) < abs(zeta):
        raise ValueError(f"Constraint violated: |ρ| must be greater than or equal to |ζ|.")
    if abs(rho) >= 1:
        raise ValueError(f"Constraint violated: |ρ| must be less than or equal to 1. Received |ρ| = {abs(rho)}.")

    a_hat = lambda eta: eta + rho + beta * zeta
    hi_hat = lambda eta: a_hat(eta) ** 2 - (1 - beta ** 2) * (rho ** 2 - zeta ** 2)
    mu_hat = lambda eta: (a_hat(eta) * np.sqrt(hi_hat(eta)) + beta * (rho ** 2 - zeta ** 2)) / (
            beta * a_hat(eta) + np.sqrt(hi_hat(eta)))


    t_hat = lambda eta: (a_hat(eta) ** 2 - (rho ** 2 - zeta ** 2)) / (beta * a_hat(eta) + np.sqrt(hi_hat(eta)))

    # # ------Direct computation of q_nu-------
    #
    # # Compute the modified Bessel functions of the second kind
    # K1 = lambda eta: besselk(nu + 1 / 2, mu_hat(eta))
    # K2 = lambda eta: besselk(nu + 3 / 2, mu_hat(eta))
    #
    # # Define the quadrature function q_nu
    # q_nu = lambda eta: (
    #         1 / (t_hat(eta) + beta * mu_hat(eta)) ** 2 *
    #         (
    #                 -1 * (rho ** 2 - zeta ** 2) * np.exp(mu_hat(eta)) * K1(eta) /
    #                 (mu_hat(eta) ** (nu + 1 / 2) * (t_hat(eta) + beta * mu_hat(eta))) +
    #
    #                 t_hat(eta) * np.exp(mu_hat(eta)) *
    #                 (K1(eta) - K2(eta)) /
    #                 (mu_hat(eta) ** (nu - 1 / 2))
    #         )
    # )

    # ------Computation of q_nu for nu=1 (Special case)-------
    # Direct computation for ν = 1 modified Bessel function of the second kind
    exp_K1 = lambda eta: np.sqrt(np.pi / (2 * mu_hat(eta))) * (1 + 1 / (2 * mu_hat(eta)))
    exp_K2 = lambda eta: np.sqrt(np.pi / (2 * mu_hat(eta))) * (1 + 3 / (2 * mu_hat(eta)) + 3 / (4 * mu_hat(eta)**2))

    # Define the quadrature function q_nu
    q_nu = lambda eta: (
            1 / (t_hat(eta) + beta * mu_hat(eta)) ** 2 *
            (
                    -1 * (rho ** 2 - zeta ** 2) * exp_K1(eta) /
                    (mu_hat(eta) ** (nu + 1 / 2) * (t_hat(eta) + beta * mu_hat(eta))) +

                    t_hat(eta) *
                    (exp_K1(eta) - exp_K2(eta)) /
                    (mu_hat(eta) ** (nu - 1 / 2))
            )
    )

    # Exponential weighting
    exp_q_nu = lambda eta: np.exp(-eta) * q_nu(eta)

    if index == 11:
        # ** Gauss-Legendre Quadrature **
        psi_nu_I = complex_quad(exp_q_nu, 0, np.abs(rho))
        psi_nu_GLeg = gauss_legendre_quadrature_ab(exp_q_nu, 0, np.abs(rho), n)

        return psi_nu_I, psi_nu_GLeg


    if index == 12:
        # ** Gauss-Laguerre Quadrature (trunted)**
        psi_nu_II = complex_quad(exp_q_nu, 1, np.inf)
        # Change of variables for Gauss-Laguerre quadrature
        q_nu_cv = lambda eta: q_nu(eta + 1) * 1 / np.exp(1)
        psi_nu_GLag = gauss_laguerre_quadrature(lambda eta: q_nu_cv(eta), n, theta)
        
        return psi_nu_II, psi_nu_GLag

    if index == 13:
        # ** hp-Gauss Quadrature **
        # Change of variables to the interval [0,1]
        exp_q_nu_cv = lambda y: (1 - np.abs(rho)) * exp_q_nu(np.abs(rho) * (1 - y) + y)
        psi_nu_III = abs(complex_quad(exp_q_nu_cv, 0, 1))
        psi_nu_hp_Gauss_GLeg, N = np.abs(hp_gauss_legendre_quadrature(exp_q_nu_cv, sigma, mu, n))

        return psi_nu_III, psi_nu_hp_Gauss_GLeg, N

***Fixing Gauss-Laguerre and $hp$-Gauss.***

In [ ]:
n_values_GLeg = np.arange(2, 25)
n_values_hp_Gauss = 45
n_values_GLag = 120

num_iterations = np.zeros((len(l_values), len(n_values_GLeg)))
relative_error = np.zeros((len(l_values), len(n_values_GLeg)))

for i, n in enumerate(n_values_GLeg):
    for j, l in enumerate(l_values):
        psi_nu_GLeg, Q_psi_nu_GLeg= compute_psi_nu_quadrature_individual(n, l, 11)

        abs_err_GLeg = np.abs(np.abs(psi_nu_GLeg) - np.abs(Q_psi_nu_GLeg))
        #----------------
        psi_nu_hp_Gauss, Q_psi_nu_hp_Gauss, N = compute_psi_nu_quadrature_individual(n_values_hp_Gauss, l, 13)
        num_iterations[j, i] = N

        abs_err_hp_Gauss = np.abs(np.abs(psi_nu_hp_Gauss) - np.abs(Q_psi_nu_hp_Gauss))
        #----------------
        psi_nu_GLag, Q_psi_nu_GLag = compute_psi_nu_quadrature_individual(n_values_GLag, l, 12)
        abs_err_GLag = np.abs(np.abs(psi_nu_GLag) - np.abs(Q_psi_nu_GLag))

        num_iterations[j, i] = N + n_values_GLag + n

        relative_error[j, i] = (abs_err_hp_Gauss + abs_err_GLeg + abs_err_GLag)/ abs(psi_nu_hp_Gauss + psi_nu_GLeg + psi_nu_GLag)


print(num_iterations[0,:])

#___________Plot___________
fig, ax = plt.subplots(1, 1, figsize=(12, 14))

plt.suptitle("Relative Error Analysis", fontsize=16)

for j, l in enumerate(l_values):
    ax.semilogy(num_iterations[j, :] ** (1/2), relative_error[j, :], label=f"l={l}")

ax.set_title(r'Semilog plot of relative error with a fixed number of support points for GLag and $hp$-Gauss') 

ax.legend()
ax.set_ylabel("Relative Error")
ax.set_xlabel(r'Total number of Quadrature Nodes$^{1/2}$')
ax.grid(True, linestyle='--', color='gray', linewidth=0.5)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

***Fixing Gauss-Legendre and Gauss-Laguerre.***

In [ ]:
n_values_GLeg = 25
n_values_hp_Gauss = np.arange(2, 45)
n_values_GLag = 120

num_iterations = np.zeros((len(l_values), len(n_values_hp_Gauss)))
relative_error = np.zeros((len(l_values), len(n_values_hp_Gauss)))


for i, n in enumerate(n_values_hp_Gauss):
    for j, l in enumerate(l_values):
        psi_nu_GLeg, Q_psi_nu_GLeg= compute_psi_nu_quadrature_individual(n_values_GLeg, l, 11)

        abs_err_GLeg = np.abs(np.abs(psi_nu_GLeg) - np.abs(Q_psi_nu_GLeg))
        #----------------
        psi_nu_hp_Gauss, Q_psi_nu_hp_Gauss, N = compute_psi_nu_quadrature_individual(n, l, 13)
        num_iterations[j, i] = N

        abs_err_hp_Gauss = np.abs(np.abs(psi_nu_hp_Gauss) - np.abs(Q_psi_nu_hp_Gauss))
        #----------------
        psi_nu_GLag, Q_psi_nu_GLag = compute_psi_nu_quadrature_individual(n_values_GLag, l, 12)
        abs_err_GLag = np.abs(np.abs(psi_nu_GLag) - np.abs(Q_psi_nu_GLag))
        # ----------------
        num_iterations[j, i] = N + n_values_GLeg + n_values_GLag

        relative_error[j, i] = (abs_err_hp_Gauss + abs_err_GLeg + abs_err_GLag)/ abs(psi_nu_hp_Gauss + psi_nu_GLeg + psi_nu_GLag)


#___________Plot___________
fig, ax = plt.subplots(1, 1, figsize=(12, 14)) 

plt.suptitle("Relative Error Analysis", fontsize=16)

for j, l in enumerate(l_values):
    ax.semilogy(num_iterations[j, :] ** (1/2), relative_error[j, :], label=f"l={l}")


ax.set_title(r'Semilog plot of relative error with a fixed number of support points for GLeg and GLag')

ax.legend()
ax.set_ylabel("Relative Error")
ax.set_xlabel(r'Total number of Quadrature Nodes$^{1/2}$')
ax.grid(True, linestyle='--', color='gray', linewidth=0.5)


plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

***Fixing Gauss-Legendre and $hp$-Gauss.***

In [ ]:
n_values_GLeg = 25                                                                                                                          
n_values_hp_Gauss = 45                                                                                                                      
n_values_GLag = np.arange(2, 120)                                                                                                           
                                                                                                                                            
num_iterations = np.zeros((len(l_values), len(n_values_GLag)))                                                                              
relative_error = np.zeros((len(l_values), len(n_values_GLag)))         

for i, n in enumerate(n_values_GLag):                                                                                                                                                                                                                              
    for j, l in enumerate(l_values):                                                                                                                                                                                           
        psi_nu_GLeg, Q_psi_nu_GLeg= compute_psi_nu_quadrature_individual(n_values_GLeg, l, 11)                                                         
                                                                                                                                            
        abs_err_GLeg = np.abs(np.abs(psi_nu_GLeg) - np.abs(Q_psi_nu_GLeg))                                                                  
        #----------------                                                                                                                   
        psi_nu_hp_Gauss, Q_psi_nu_hp_Gauss, N = compute_psi_nu_quadrature_individual(n_values_hp_Gauss, l, 13)                                         
        num_iterations[j, i] = N                                                                                                            
                                                                                                                                            
        abs_err_hp_Gauss = np.abs(np.abs(psi_nu_hp_Gauss) - np.abs(Q_psi_nu_hp_Gauss))                                                      
        #----------------                                                                                                                   
        psi_nu_GLag, Q_psi_nu_GLag = compute_psi_nu_quadrature_individual(n, l, 12)                                                                    
        abs_err_GLag = np.abs(np.abs(psi_nu_GLag) - np.abs(Q_psi_nu_GLag))                                                                  
                                                                                                                                            
        num_iterations[j, i] = N + n_values_GLeg + n                                                                                        
                                                                                                                                            
        relative_error[j, i] = (abs_err_hp_Gauss + abs_err_GLeg + abs_err_GLag)/ abs(psi_nu_hp_Gauss + psi_nu_GLeg + psi_nu_GLag)           
                                                                                                                                                                                                                                                     
                                                                                                                                            
#___________Plot___________                                                                                                           
                                                                                                                                            
ref = lambda x: 10**10 * np.exp(-2/3 * x ** (1/2))                                                                                          
fig, ax = plt.subplots(1, 1, figsize=(12, 14))                                                              
                                                                                                                                            
plt.suptitle("Relative Error Analysis", fontsize=16)                                                                                        
                                                                                                                                            
for j, l in enumerate(l_values):                                                                                                            
    ax.semilogy(num_iterations[j, :] ** (1/2), relative_error[j, :], label=f"l={l}")                                                        
                                                                                                                                                                                                        
ax.set_title(r'Semilog plot of relative error with a fixed number of support points for GLeg and $hp$-Gauss')            
                                                                                                                                            
ax.legend()                                                                                                                                 
ax.set_ylabel("Relative Error")                                                                                                             
ax.set_xlabel(r'Total number of Quadrature Nodes$^{1/2}$')                                                                                  
ax.grid(True, linestyle='--', color='gray', linewidth=0.5)                                                                                  
                                                                                                                                                                            
                                                                                                                                            
plt.tight_layout(rect=[0, 0, 1, 0.95])                                                                                                      
plt.show()                  

### Comparison of the Splitting Method and Gauss-Laguerre Quadrature

Comparison of the accuracy and efficiency of the splitting method versus Gauss-Laguerre quadrature.

Initialize arrays to store results

In [ ]:
num_iterations = np.zeros((len(l_values), len(n_values)))
relative_error_splitting = np.zeros((len(l_values), len(n_values)))
relative_error_GLag = np.zeros((len(l_values), len(n_values)))

Compute errors for each combination of $n$ and $l$

In [ ]:
for j, l in enumerate(l_values):
    print(f"----------l={l}---------------")
    for i, n in enumerate(n_values):
        print(f"splitting methode n={n}")
        psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)

        # Compute the total number of quadrature nodes
        num_iterations[j, i] = 2 * n + N

        # Compute relative error for the splitting method
        relative_error_splitting[j, i] = np.abs(np.abs(psi_nu) - np.abs(Q_psi_nu)) / np.abs(psi_nu)

    # Compute Gauss-Laguerre error for corresponding n values
    for i, n in enumerate(num_iterations[j, :]):
        print(f"mono methode n={n}")
        psi_nu_GLag = compute_psi_nu_quadrature(int(n), l, 1)

        if np.isnan(psi_nu_GLag):
            print(f"WARNING: NaN detected in Gauss-Laguerre for n={n}, l={l}")
            relative_error_GLag[j, i] = np.nan
        else:
            relative_error_GLag[j, i] = np.abs(np.abs(psi_nu) - np.abs(psi_nu_GLag)) / np.abs(psi_nu)

# Replace NaN values with the maximum valid error to ensure proper plotting
relative_error_GLag = np.nan_to_num(relative_error_GLag, nan=np.nanmax(relative_error_GLag[~np.isnan(relative_error_GLag)]))


Plot the relative error values for comparison

In [ ]:
length = math.ceil(len(l_values) /2)

fig, axs = plt.subplots(2, length, figsize=(18, 10))
plt.suptitle("Relative Error Comparison: Splitting Method vs. Gauss-Laguerre", fontsize=16)

# Plot for the first 5 l_values (row 0)
for j, l in enumerate(l_values[:length]):
    axs[0, j].semilogy(num_iterations[j, :] ** (1/2), relative_error_splitting[j, :], linewidth=3)
    axs[0, j].semilogy(num_iterations[j, :]** (1/2), relative_error_GLag[j, :], linewidth=3)
    axs[0, j].set_title(f"l = {l}")

# Plot for the next 5 l_values (row 1)
for j, l in enumerate(l_values[length:]):
    axs[1, j].semilogy(num_iterations[j + length, :]** (1/2), relative_error_splitting[j + length, :], linewidth=3)
    axs[1, j].semilogy(num_iterations[j + length, :]** (1/2), relative_error_GLag[j + length, :], linewidth=3)
    axs[1, j].set_title(f"l = {l}")
axs[1, 0].set_ylabel("Relative Error")

plt.subplots_adjust(hspace=0.4, wspace=0.3)
legend = fig.legend(["Splitting Method", "Gauss-Laguerre"], loc="upper center", bbox_to_anchor=(0.5, 0.95), ncol=2, fontsize=10)

for ax in axs.flat:
    ax.set_xlabel(r'Number of Quadrature Nodes$^{1/2}$')
    ax.grid(True, linestyle='--', color='lightgray', linewidth=0.1)


plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


### Computation of the Constant for the Error Estimate

Calculates the constant used in defining the upper bound function, ensuring accurate estimations and stability in numerical analysis.

In [ ]:
# Define parameter ranges
n_values = np.arange(1, 10)  # Number of quadrature nodes
l_values = np.arange(1, 6)  # Values for exponent l

#### Gauss-Legendre quadrature 

The approximation using Gauss-Legendre quadrature on the interval $(0, |\rho|)$, based on the formula
\begin{equation} 
\left|\psi_{\nu}^{\text{GLeg}}(\rho, \zeta) - Q_n^{\text{GLeg}} \right| \leq \kappa^{-2n} |\rho|^{-2\nu-1} \, \left( 1 - \frac{1}{2} |\rho| \right)^{-2 \nu - 2} 
\end{equation} 
Since the remaining term 
$$
R_n^{\text{GLeg}} := \left|\psi_{\nu}^{\text{GLeg}}(\rho, \zeta) - Q_n^{\text{GLeg}} \right|
$$
can be determined using the quadrature method, the constant $\kappa$ can be directly computed as follows:

$$
\kappa = \left( R_n^{\text{GLeg}} \cdot |\rho|^{2\nu +1} \cdot \left( 1 - \frac{1}{2} |\rho| \right)^{2 \nu + 2} \right)^{-\frac{1}{2n}}
$$

This direct computation ensures a precise estimation of $\kappa$.


In [ ]:
def compute_kappa(n_values, l_values):
    """
    Computes the constant kappa for given quadrature points n_values and exponents l_values.

    Parameters:
    -----------
    n_values : array-like
        Array of quadrature nodes (number of points used in the method).
    l_values : array-like
        Array of exponent values controlling rho.

    Returns:
    --------
    kappa_list : numpy.ndarray
        Array of computed kappa values for each l in l_values.
    kappa_mean : float
        Mean of all computed kappa values (ignoring NaNs).
    """

    kappa_list = []  
    
    # Compute kappa for each l and n
    for l in l_values:
        rho = 2.0 ** (-l)

        kappa_values_n = [] 

        for n in n_values:
            # Compute psi_nu and quadrature approximations
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)

            exact_error = np.abs(psi_nu_vec[0] - Q_psi_nu_vec[0])
            kappa = (exact_error * np.abs(rho) ** (2 * nu + 1) * (1 - 0.5 * np.abs(rho)) ** (2 * nu + 2)) ** (-1 / (2 * n))
            kappa_values_n.append(kappa)

        if kappa_values_n:
            kappa_list.append(np.mean(kappa_values_n))
        else:
            # Handle cases where no valid kappa was found
            kappa_list.append(np.nan)

    kappa_list = np.array(kappa_list)

    # Compute the overall mean of kappa values (ignoring NaN values)
    kappa_mean = np.nanmean(kappa_list)

    # Print results
    print("\n=== Computed Constant kappa ===")
    print(f"Total computed kappa values: {len(kappa_list)}")
    print(f"Mean kappa value: {kappa_mean:.4f}")
    
    return kappa_list, kappa_mean

Example function call

In [ ]:
kappa_list, kappa_mean = compute_kappa(n_values, l_values)

print(f"kappa = {[f'{kappa:.3f}' for kappa in kappa_list]}")

#### Gauss-Legendre quadrature on a Geometrically Graded Mesh 
**($hp$-Gauss)**

The approximation using Gauss-Legendre quadrature on a geometrically graded mesh on the interval $(|\rho|, 1)$, defined by the bound
\begin{equation} 
\left|\psi_{\nu}^{\text{hp}}(\rho, \zeta) - Q_n^{\text{hp}} \right| \leq C  (1-|\rho|)^{-2\nu -1} \left( \frac{2}{1 + |\rho|} \right)^{2\nu +2} \text{e}^{-bN^{1/2}}
\end{equation} 
where $C$ and $b$ are positive constants that need to be determined and $N$ denotes the total number of function evaluations for $n$ nodes.

Since the remaining term 
$$
R_{N,\sigma}^{\text{hp}} := \left|\psi_{\nu}^{\text{hp}}(\rho, \zeta) - Q_n^{\text{hp}} \right|
$$
the constants $b$ and $C$ can be determined using:
\begin{equation}
\begin{split}
b &= \frac{\ln\left(R_{N_2,\sigma}^{\text{hp}}\right) - \ln\left(R_{N_1,\sigma}^{\text{hp}}\right)}{N_2^{1/2} - N_1^{1/2}}, \\[11 pt]
C &= \frac{R_{N_1,\sigma}^{\text{hp}}}{(1-|\rho|)^{-2\nu -1} \left( \frac{2}{1 + |\rho|} \right)^{2\nu +2} \text{e}^{-b N_1^{1/2}}}.
\end{split}
\end{equation}
Here, the indices 1 and 2 indicate two different choices of  $N$, allowing for an empirical estimation of the constants.


In [ ]:
def compute_C_b_hp_gauss(n_values, l_values, gamma):
    """
    Computes the constants C and b for the upper bound function.

    Parameters:
    -----------
    n_values : array-like
        Array of quadrature nodes (number of points used in the method).
    l_values : array-like
        Array of exponent values controlling rho.
    gamma : float, optional
        Exponent parameter (default is 2).

    Returns:
    --------
    C_list : list
        List of computed C values for each l.
    b_list : list
        List of computed b values for each l.
    C_mean : float
        Mean value of C across computations.
    b_mean : float
        Mean value of b across computations.
    """

    C_list, b_list, rho_list = [], [], []
    
    A = lambda rho: (1 - np.abs(rho)) ** (-2 * nu - 1) * (2 / (1 + np.abs(rho))) ** (2 * nu + 2)

    # Increase l by 1 to achieve a more stable error reduction.
    # Smaller values of rho = 2^(-l) improve numerical approximation 
    # and lead to a more accurate computation of the constants C and b.
    l_values = [l + 1 for l in l_values]
    
    for l in l_values:
        rho = 2.0 ** (-l)
        exact_err_l, N_l = [], []
        b_rho = []

        index = 0

        psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n_values[index], l, 0)
        N_l.append(N)
        exact_err_l.append(np.abs(psi_nu_vec[2] - Q_psi_nu_vec[2]))

        for n in n_values[index:]:
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)

            if N not in N_l and np.sqrt(N) < 20:
                N_l.append(N)
                exact_err_l.append(np.abs(psi_nu_vec[2] - Q_psi_nu_vec[2]))

                if len(N_l) > 7:
                    err1, err2 = exact_err_l[2], exact_err_l[-1]
                    N1, N2 = N_l[2], N_l[-1]
                    
                    if err1 != err2 and N1 != N2:
                        b = np.log(err2 / err1) / (N2 ** (1 / gamma) - N1 ** (1 / gamma))
                        b_rho.append(b)
        if b_rho:
            b_rho_mean = np.mean(np.abs(b_rho))
            b_list.append(b_rho_mean)
        else:
            b_list.append(np.nan)

        C_rho_list = [
            exact_err_l[i] / (A(rho) * np.exp(-b_rho_mean * N ** (1 / gamma)))
            for i, N in enumerate(N_l)
        ]

        if C_rho_list:
            C_list.append(np.mean(C_rho_list))
        else:
            C_list.append(np.nan)


        
    C_list = np.array(C_list, dtype=np.float64).ravel()
    b_list = np.array(b_list, dtype=np.float64).ravel()

    # Compute overall mean of C and  b values
    C_mean = np.nanmean(C_list)
    b_mean = np.nanmean(b_list)    

    # Compute 95% confidence interval for C and b
    C_lower_bound, C_upper_bound = np.percentile(C_list, [2.5, 97.5])
    b_lower_bound, b_upper_bound = np.percentile(b_list, [2.5, 97.5])

    # Print results
    print("\n=== Computed Constants C and b ===")
    print(f"Total computed b values: {len(b_list)}")
    print(f"Total computed C values: {len(C_list)}")
    print(f"Mean C: {C_mean:.4f}, Mean b: {b_mean:.4f}")
    print(f"95% Confidence Interval for b: [{b_lower_bound:.4f}, {b_upper_bound:.4f}]")
    print(f"95% Confidence Interval for C: [{C_lower_bound:.4f}, {C_upper_bound:.4f}]")

    return C_list, b_list, C_mean, b_mean

Example function call

In [ ]:
C_list, b_list, C_mean, b_mean = compute_C_b_hp_gauss(n_values, l_values , 2)

print(f"C = {[f'{c:.3f}' for c in C_list]}")
print(f"b = {[f'{b:.3f}' for b in b_list]}")

#### Gauss-Laguerre quadrature

The approximation using Gauss-Laguerre quadrature on the interval $(1, \infty)$, described by the inequality
\begin{equation} 
\left|\psi_{\nu}^{\text{GLag}}(\rho, \zeta) - Q_n^{\text{GLag}} \right| \leq C \cdot 2.1 \cdot \left( \frac{2}{1 + |\rho|} \right)^{\nu + 1} \cdot \sqrt{2\pi} \cdot \mathrm{e}^{-\alpha \cdot \sqrt[\gamma]{\frac{1 + |\rho|}{2} \cdot n}}
\end{equation}

here $C$, $\alpha$ and $\gamma$ are positive constants that need to be determined and $n$ denotes the total number of quadrature nodes.

To estimate these constants, we consider the remaining term remaining error term, defined as:
$$
R_{n}^{\text{GLag}} := \left|\psi_{\nu}^{\text{GLag}}(\rho, \zeta) - Q_n^{\text{GLag}} \right|
$$
the constants $C$, $\alpha$ and $\gamma$ can be determined by solving the following nonlinear system of equations:
\begin{equation}
\begin{cases}
C \cdot 2.1 \cdot \left( \frac{2}{1 + |\rho|} \right)^{\nu + 1} \cdot \sqrt{2\pi} \cdot \mathrm{e}^{-\alpha \cdot \sqrt[\gamma]{\frac{1 + |\rho|}{2} \cdot n_1}} - |R_{n_1}^{\text{GLag}}| = 0, \\[11 pt]
C \cdot 2.1 \cdot \left( \frac{2}{1 + |\rho|} \right)^{\nu + 1} \cdot \sqrt{2\pi} \cdot \mathrm{e}^{-\alpha \cdot \sqrt[\gamma]{\frac{1 + |\rho|}{2} \cdot n_2}} - |R_{n_2}^{\text{GLag}}| = 0, \\[11 pt]
C \cdot 2.1 \cdot \left( \frac{2}{1 + |\rho|} \right)^{\nu + 1} \cdot \sqrt{2\pi} \cdot \mathrm{e}^{-\alpha \cdot \sqrt[\gamma]{\frac{1 + |\rho|}{2} \cdot n_3}} - |R_{n_3}^{\text{GLag}}| = 0.
\end{cases}
\end{equation}
Here, the indices 1, 2 and 3 indicate different choices of  $n$, allowing for an estimation of the constants based on numerical data.

To solve the given nonlinear system of equations efficiently, the **Newton method** is the most suitable approach. This iterative numerical method enables us to determine the optimal value of the exponent $\gamma$ by minimizing the residual errors in the system. Once the best choice for $\gamma$ has been obtained, the constants $C$ and $\alpha$ can be explicitly determined through direct computation.

The expressions for $\alpha$ and $C$ are given by:
\begin{equation}
\begin{split}
\alpha &= \frac{\ln\left(|R_{n_2}^{\text{GLag}}|\right) - \ln\left(|R_{n_1}^{\text{GLag}}|\right)}
{\sqrt[\gamma]{\frac{1 + |\rho|}{2}} \left( n_1^{1/\gamma} - n_2^{1/\gamma} \right)}, \\[11 pt]
C &= \frac{|R_{n_2}^{\text{GLag}}|}{2.1 \cdot \left( \frac{2}{1 + |\rho|} \right)^{\nu + 1} \cdot \sqrt{2\pi} \cdot 
\mathrm{e}^{-\alpha \cdot \left(\frac{1 + |\rho|}{2} \cdot n_1 \right)^{1/\gamma} }}.
\end{split}
\end{equation}

To avoid small values that could affect the solution of the Newton method, define a program that filters elements from an array until it encounters a value below a specified threshold and returns the filtered list along with its length.

In [ ]:
def filter_until_threshold(array, threshold=1e-14):
    """
    Filters elements from the input array until an element below the threshold is encountered.

    Parameters:
    -----------
    array : iterable
        Input array or list of numerical values.
    threshold : float, optional
        The stopping condition; elements below this value will not be included (default: 1e-14).

    Returns:
    --------
    filtered_list : list
        List of elements up to the first element below the threshold.
    count : int
        Number of elements in the filtered list.
    """
    filtered_list = [element for element in array if element >= threshold]

    return filtered_list, len(filtered_list)

In [ ]:
def inverse_3x3(A, l):
    """
    Computes the inverse of a 3x3 matrix using the adjugate method.

    Parameters:
    -----------
    A : numpy.ndarray
        A 3x3 NumPy array representing the matrix to be inverted.
    l : int or float
        Parameter used for error reporting if the matrix is singular.

    Returns:
    --------
    Inv_A : numpy.ndarray
        The inverse of the input matrix A.

    Raises:
    -------
    ValueError:
        If the determinant is zero, indicating that the matrix is singular.
    """

    # Compute the determinant of A
    det_A = (
        A[0, 0] * (A[1, 1] * A[2, 2] - A[1, 2] * A[2, 1])
        - A[0, 1] * (A[1, 0] * A[2, 2] - A[1, 2] * A[2, 0])
        + A[0, 2] * (A[1, 0] * A[2, 1] - A[1, 1] * A[2, 0])
    )

    # Check if the matrix is singular
    if det_A == 0:
        raise ValueError(f"The Jacobian matrix is singular. Newton's method cannot be applied for l = {l}.")

    # Compute the adjugate (cofactor) matrix
    adjoint_A = np.array([
        [ A[1, 1] * A[2, 2] - A[1, 2] * A[2, 1], -(A[0, 1] * A[2, 2] - A[0, 2] * A[2, 1]),  A[0, 1] * A[1, 2] - A[0, 2] * A[1, 1]],
        [-(A[1, 0] * A[2, 2] - A[1, 2] * A[2, 0]),  A[0, 0] * A[2, 2] - A[0, 2] * A[2, 0], -(A[0, 0] * A[1, 2] - A[0, 2] * A[1, 0])],
        [ A[1, 0] * A[2, 1] - A[1, 1] * A[2, 0], -(A[0, 0] * A[2, 1] - A[0, 1] * A[2, 0]),  A[0, 0] * A[1, 1] - A[0, 1] * A[1, 0]]
    ])

    # Compute the inverse matrix
    Inv_A = adjoint_A / det_A

    return Inv_A

In [ ]:
# Defines C, alpha, and gamma as symbolic variables for symbolic computation in SymPy.
C, alpha, gamma = sp.symbols('C alpha gamma') 

Implements Newton's method to solve the nonlinear equation system defined above.

In [ ]:
def newton_method(tolerance, f, x0, l):
    """
    Implements Newton's method for solving a system of nonlinear equations.

    Parameters:
    -----------
    tolerance : float
        Convergence criterion; the algorithm stops when the difference between iterations is below this value.
    f : sympy.Matrix
        The system of nonlinear equations to be solved.
    x0 : numpy.ndarray
        Initial guess for the solution.
    l : int or float
        Parameter used for error handling and reporting.

    Returns:
    --------
    x_solution : numpy.ndarray
        The solution vector obtained using Newton's method.
    iterations : int
        The number of iterations performed before convergence.

    Raises:
    -------
    ValueError:
        If the maximum number of iterations is reached without convergence.
    """

    # Compute the Jacobian matrix of f with respect to C, alpha, and gamma
    jacobian_f = f.jacobian([C, alpha, gamma])

    # Initialize iteration variables
    x_iteration = x0
    iteration = 0
    max_iterations = 100  # Limit for iterations
    difference = tolerance + 1

    while difference > tolerance and iteration < max_iterations:
        f_x_iteration = np.array(f.subs({C: x_iteration[0], alpha: x_iteration[1], gamma: x_iteration[2]}),
                                 dtype=float).flatten()

        jacobian_x_iteration = np.array(
            jacobian_f.subs({C: x_iteration[0], alpha: x_iteration[1], gamma: x_iteration[2]}), dtype=float)
        inv_jacobian_x_iteration = inverse_3x3(jacobian_x_iteration, l)

        #x_new = x_iteration - inv_jacobian_x_iteration @ f_x_iteration

        # Alternative approach using NumPy's linear solver
        x_new = x_iteration - np.linalg.solve(jacobian_x_iteration, f_x_iteration)

        difference = np.linalg.norm(x_iteration - x_new)

        x_iteration = x_new
        iteration += 1

    # Check for non-convergence
    if iteration >= max_iterations:
        raise ValueError(f"Newton's method did not converge within {max_iterations} iterations for l = {l}.")

    return x_iteration, iteration

Computes the constants $C$, $\alpha$, and $\gamma$ using Newton's method by solving a nonlinear system.

In [ ]:
def compute_constants_via_newton(n_values, l_values, tolerance=1e-12):
    """
        Computes the constants C, alpha, and gamma using Newton's method by solving a nonlinear system.

        Parameters:
        -----------
        n_values : array-like
            List of quadrature node counts.
        l_values : array-like
            List of l values for which the computation is performed.
        tolerance : float, optional
            Convergence tolerance for Newton's method (default: 1e-12).

        Returns:
        --------
        C_alpha_gamma_means : numpy.ndarray
            Array containing the mean values of C, alpha, and gamma over all computations.
        """


    # Initialize list to store computed values of (C, alpha, gamma)
    C_alpha_gamma_values_Newton = []

    # Loop over l values
    for l in l_values:
        rho = 2.0 ** (-l)

        exact_n = np.empty(len(n_values))

        # Compute remainder term values
        for k, n in enumerate(n_values):
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)
            exact_n[k] = np.abs(psi_nu_vec[1] - Q_psi_nu_vec[1])

        # Filter values until threshold
        exact_n_new, n_index = filter_until_threshold(exact_n, threshold=1e-14)
        n_new = np.arange(1, n_index + 1)

        # Select three values of n for the nonlinear system
        n1, exact_n1 = n_new[1], exact_n_new[1]
        n2, exact_n2 = n_new[-3], exact_n_new[-3]
        k = math.ceil((n_index + 1) / 2)
        n3, exact_n3 = n_new[k], exact_n_new[k]

    # Define the nonlinear system using SymPy
        f = sp.Matrix([
            C * 2.1 * (2 / (1 + sp.Abs(rho))) ** (nu + 1) * sp.sqrt(2 * sp.pi) * sp.exp(
                -alpha * ((1 + sp.Abs(rho)) / 2 * n1) ** (1 / gamma)) - exact_n1,
            C * 2.1 * (2 / (1 + sp.Abs(rho))) ** (nu + 1) * sp.sqrt(2 * sp.pi) * sp.exp(
                -alpha * ((1 + sp.Abs(rho)) / 2 * n2) ** (1 / gamma)) - exact_n2,
            C * 2.1 * (2 / (1 + sp.Abs(rho))) ** (nu + 1) * sp.sqrt(2 * sp.pi) * sp.exp(
                -alpha * ((1 + sp.Abs(rho)) / 2 * n3) ** (1 / gamma)) - exact_n3
        ])

        # Initial guess for Newton's method
        x0 = np.array([0.5, 2.5, 1.8])

        # Apply Newton's method to solve for (C, alpha, gamma)
        x_Newton, num_iterations_Newton = newton_method(tolerance, f, x0, l)

        C_alpha_gamma_values_Newton.append(x_Newton)

    # Compute the mean values of C, alpha, and gamma
    C_alpha_gamma_values_Newton = np.array(C_alpha_gamma_values_Newton, dtype=np.float64)
    C_alpha_gamma_means = np.mean(C_alpha_gamma_values_Newton, axis=0)

    # Print results
    print(f"Newton: The approximated average root is: (C, alpha, gamma) = ({C_alpha_gamma_means[0]:.4f}, {C_alpha_gamma_means[1]:.4f}, {C_alpha_gamma_means[2]:.4f}) with tolerance {tolerance}")

    return C_alpha_gamma_means

Determination of the constants $C$ and $\alpha$ for the optimal choice of $\gamma$ using Newton's method.

In [ ]:
def compute_C_alpha_glag(n_values, l_values, gamma):
    C_list, alpha_list, rho_list = [], [], []

    A = lambda rho: 2.1 * (2 / (1 + np.abs(rho)) ** (nu + 1) * np.sqrt(2 * np.pi))

    for l in l_values:
        rho = 2.0 ** (-l)
        exact_err_l, n_l = [], []
        alpha_rho = []

        index = 2

        psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n_values[index], l, 0)
        n_l.append(n_values[index])
        exact_err_l.append(np.abs(psi_nu_vec[1] - Q_psi_nu_vec[1]))

        for n in n_values[index + 1:]:
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)
            n_l.append(n)
            exact_err_l.append(np.abs(psi_nu_vec[1] - Q_psi_nu_vec[1]))

            err1, err2, n1, n2 = None, None, None, None
            if len(n_l) > 4:
                err1, err2 = exact_err_l[2], exact_err_l[-1]
                n1, n2 = n_l[2], n_l[-1]

            if err1 is not None and err2 is not None and n1 is not None and n2 is not None:
                if err1 != err2 and n1 != n2:
                    numerator = np.log(err2) - np.log(err1)
                    denominator = ((1 + np.abs(rho)) / 2 * n1) ** (1 / gamma) - ((1 + np.abs(rho)) / 2 * n2) ** (1 / gamma)
                    alpha_rho = numerator / denominator

        # Compute mean alpha_rho value
        alpha_rho_mean = np.nan if not alpha_rho else np.mean(np.abs(alpha_rho))
        alpha_list.append(alpha_rho_mean)

        C_rho_list = [ exact_err_l[i] / (A(rho) * np.exp(-alpha_rho_mean * ((1 + np.abs(rho)) / 2 * n) ** (1 / gamma))) for i, n in enumerate(n_l)]

        if C_rho_list:
            C_list.append(np.mean(C_rho_list))
        else:
            C_list.append(np.nan)

    C_list = np.array(C_list, dtype=np.float64).ravel()
    alpha_list = np.array(alpha_list, dtype=np.float64).ravel()

    # Compute overall mean of C and  b values
    C_mean = np.nanmean(C_list)
    alpha_mean = np.nanmean(alpha_list)

    # Compute 95% confidence interval for C and b
    C_lower_bound, C_upper_bound = np.percentile(C_list, [2.5, 97.5])
    alpha_lower_bound, alpha_upper_bound = np.percentile(alpha_list, [2.5, 97.5])

    # Print results
    print("\n=== Computed Constants C and b ===")
    print(f"Total computed b values: {len(alpha_list)}")
    print(f"Total computed C values: {len(C_list)}")
    print(f"Mean C: {C_mean:.4f}, Mean b: {alpha_mean:.4f}")
    print(f"95% Confidence Interval for b: [{alpha_lower_bound:.4f}, {alpha_upper_bound:.4f}]")
    print(f"95% Confidence Interval for C: [{C_lower_bound:.4f}, {C_upper_bound:.4f}]")

    return C_list, alpha_list, C_mean, alpha_mean

Example function call:

**Remark:** The Newton method can run into issues when working with extremely small numbers. This can cause Jupyter to struggle with calculations, leading to `NaN` values because the numbers are too tiny to be processed accurately. Because of this, I simply use the previously used value for `gamma` to execute the code here.

In [ ]:
#test = compute_constants_via_newton(n_values, l_values)

#if not np.isnan(test[2]):  # Falls kein NaN
#    gamma = math.ceil(test[2] * 10) / 10
#else:
    #print("Warning: gamma is NaN, using default value 1.5")  
    #gamma = 1.5

gamma = 1.5
C_list, alpha_list, C_mean, alpha_mean = compute_C_alpha_glag(n_values, l_values , gamma)

print(f"gamma = {gamma: .3f}")
print(f"C = {[f'{c:.3f}' for c in C_list]}")
print(f"alpha = {[f'{alpha:.3f}' for alpha in alpha_list]}")

### How expensive is the Effort?

#### Time effort

The following code measures the execution time required to evaluate the quadrature of the function 
$\psi_\nu$ for different number quadrature nodes $n$ and exponent values $l$. The results are visualized in a plot for better interpretation.

In [ ]:
def compute_execution_time(f, n_values, l_values):
    """
    Computes the execution time required to evaluate the function `f(n, l)`
    for different values of `n` (number of quadrature points) and `l` (exponent controlling rho).

    Parameters:
    -----------
    f : function
        Function that computes psi_nu and quadrature values.
    n_values : array-like
        Array of quadrature points.
    l_values : array-like
        Array of exponent values for rho.

    Returns:
    --------
    time_list : numpy.ndarray
        2D array storing computation times for each (n, l) pair.
    iteration_points : numpy.ndarray
        2D array storing the number of iteration points used.
    """

    num_l = len(l_values)
    num_n = len(n_values)

    time_list = np.zeros((num_l, num_n))
    iteration_points = np.zeros((num_l, num_n))

    
    for i, n in enumerate(n_values):
        for j, l in enumerate(l_values):
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = f(n, l, 0)
            time_list[j, i] = time_result
            iteration_points[j, i] = 2 * n + N  

    return time_list, iteration_points

 Plots the execution time as a function of the number of quadrature nodes.


In [ ]:
# Run the computation
time_list, iteration_points = compute_execution_time(compute_psi_nu_quadrature, n_values, l_values)

fig, ax = plt.subplots(figsize=(10, 6))

# Define colormap for different l values
winter_cmap = cm.get_cmap('winter', len(l_values) + 1)  # Blue-Green gradient for time lines
red_cmap = cm.get_cmap('RdPu', 5)  # Red-Purple for reference lines

for j, l in enumerate(l_values):
    ax.plot(iteration_points[j, :], time_list[j, :], label=f"$l={l}$", color=winter_cmap(j / len(l_values)))

# Sort the execution times for reference line calculation
time_sorted_values = np.sort(time_list.flatten())  
time1, time2, time_max = time_sorted_values[0], time_sorted_values[len(time_sorted_values) // 2], time_sorted_values[-1]


# Find indices and corresponding quadrature nodes
n1, n2 = [iteration_points[np.where(time_list == t)] for t in (time1, time2)]

# Define iteration range
iteration_list = np.arange(iteration_points[1:, 1:].min(), iteration_points[1:, 1:].max() + 1)

# Compute linear reference line parameters
m, q = (time2 - time1) / (n2 - n1), (n2 * time1 - n1 * time2) / (n2 - n1) - 0.02

print(f"Reference line slope (m): {m}\nReference line intercept (q): {q}")

# Reference line settings
reference_settings = [(m, 1.0, q, time_max, r'm \cdot n + q', red_cmap(2))]


# Compute and plot reference line
for factor, exponent, offset, upper_bound, label, color in reference_settings:
    f_of_n = factor * iteration_list ** exponent + offset
    valid_indices = np.where((f_of_n >= 0) & (f_of_n <= upper_bound))[0]
    n_values_filtered = iteration_list[valid_indices]
    f_of_n_filtered = f_of_n[valid_indices]

    ax.plot(n_values_filtered, f_of_n_filtered,
            label=fr"ref: ${label}$",
            color=color, linestyle="dashed")

ax.set_title("Execution Time vs. Number of Quadrature Nodes")
ax.set_xlabel("Number of Quadrature Nodes")
ax.set_ylabel("Time (seconds)")
ax.legend()

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#### Iteration Cost

The following code computes the number of quadrature points the theoretical and practical effort for quadratures of the function $\psi_\nu$ for different quadrature nodes $n$ and exponent values $l$. The results are visualized in a plot for better interpretation.

In [ ]:
def plot_comparison_effort_gauss_legendre(n_values, l_values, exponent_max, kappa_list, compute_psi_nu_quadrature):
    """
    Computes and plots the theoretical and practical effort for Gauss-Legendre quadrature.

    Parameters:
    -----------
    n_values : array-like
        Number of quadrature points.
    l_values : array-like
        List of exponent values l.
    exponent_max : int
        Maximum exponent for the error tolerance.
    kappa_list : array-like
        List of kappa values for different l.
    compute_psi_nu_quadrature : function
        Function to compute psi_nu and quadrature values.
    """
    # Define epsilon function and theoretical node estimation
    epsilon = lambda x: 10 ** (-x)
    nodes_number_GLeg = lambda rho, epsilon, kappa: (1 / (2 * np.log(kappa))) * (
        np.log(9) - (2 * nu + 1) * np.log(np.abs(rho))
        - (2 * nu + 2) * np.log(1 - 0.5 * np.abs(rho))
        - np.log(epsilon)
    )

    # Initialize storage matrices
    n_GLeg_values = np.zeros((exponent_max, len(l_values)))
    for i in range(0, exponent_max + 1):
        for j, l in enumerate(l_values):
            rho = 2.0 ** (-l)
            eps_value = epsilon(i)
            n_GLeg_values[i - 1, j] = np.ceil(nodes_number_GLeg(rho, eps_value, kappa_list[j]))

    Y_epsilon = np.array([epsilon(x) for x in range(0, exponent_max)])
    absolut_error_GLeg = np.zeros((len(l_values), len(n_values)))

    for i, n in enumerate(n_values):
        for j, l in enumerate(l_values):
            # Compute psi_nu and quadrature approximations
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)
            # Compute absolute error
            absolut_error_GLeg[j, i] = np.abs(np.abs(psi_nu_vec[0]) - np.abs(Q_psi_nu_vec[0]))

    print("Shape of n_GLeg_values[:, j]:", n_GLeg_values[:, j].shape)
    print("Shape of Y_epsilon:", Y_epsilon.shape)

    # Plot results
    length = math.ceil(len(l_values) / 2)
    fig, axs = plt.subplots(2, length, figsize = (18.5, 8.9))

    if length == 1:
        axs = axs.flatten()

    plt.suptitle("Theoretical and Practical Effort for Node Points Gauss-Legendre", fontsize=15)

    for j, l in enumerate(l_values[:length]):
        axs[0, j].semilogy(n_values, absolut_error_GLeg[j, :], label=f"Practical")
        axs[0, j].semilogy(n_GLeg_values[:, j], Y_epsilon, marker='o', linestyle='-', label=f"Theoretical")
        axs[0, j].set_title(f"l = {l}")
        axs[0, j].set_xlabel(r'Number of Quadrature Nodes', fontsize=8)
        axs[0, j].set_ylim(None, 10 ** 0 + 1)
    axs[0, 0].set_ylabel("Absolute Error")

    for j, l in enumerate(l_values[length:]):
        axs[1, j].semilogy(n_values, absolut_error_GLeg[length + j, :], label=f"Practical")
        axs[1, j].semilogy(n_GLeg_values[:, length + j], Y_epsilon, marker='o', linestyle='-', label=f"Theoretical")
        axs[1, j].set_title(f"l = {l}")
        axs[1, j].set_xlabel(r'Number of Quadrature Nodes', fontsize=8)
        axs[1, j].set_ylim(None, 10 ** 0 + 1)
    axs[1, 0].set_ylabel("Absolute Error")

    plt.subplots_adjust(hspace=0.4, wspace=0.3)
    legend = fig.legend(["Practical", "Theoretical"], loc="upper center", bbox_to_anchor=(0.5, 0.95), ncol=2, fontsize=10)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

In [ ]:
def plot_comparison_effort_hp_gauss(n_values, l_values, exponent_max, C_list, b_list, compute_psi_nu_quadrature):
    """
    Computes and plots the theoretical and practical effort for hp-Gauss-Laguerre quadrature.

    Parameters:
    -----------
    n_values : array-like
        Number of quadrature points.
    l_values : array-like
        List of exponent values l.
    exponent_max : int
        Maximum exponent for the error tolerance.
    C_list : array-like
        List of C values for different l.
    b_list : array-like
        List of b values for different l.
    compute_psi_nu_quadrature : function
        Function to compute psi_nu and quadrature values.
    """
    # Define epsilon function and theoretical node estimation
    epsilon = lambda x: 10 ** (-x)
    nodes_number_hp_Gauss = lambda rho, epsilon, C, b: (
        (np.log(C) + (-2 * nu - 1) * np.log(1 - np.abs(rho))
        + (2 * nu + 2) * np.log(2 / (1 + np.abs(rho))) - np.log(epsilon)) / b
    ) ** 2

    # Initialize storage matrices
    n_hp_Gauss_values = np.zeros((exponent_max, len(l_values)))
    for i in range(0, exponent_max + 1):
        for j, l in enumerate(l_values):
            rho = 2.0 ** (-l)
            eps_value = epsilon(i)
            n_hp_Gauss_values[i - 1, j] = np.ceil(nodes_number_hp_Gauss(rho, eps_value, C_list[j], b_list[j]))

    Y_epsilon = np.array([epsilon(x) for x in range(0, exponent_max)])
    num_iterations = np.zeros((len(l_values), len(n_values)))
    absolut_error_hp_Gauss = np.zeros((len(l_values), len(n_values)))

    for i, n in enumerate(n_values):
        for j, l in enumerate(l_values):
            # Compute psi_nu and quadrature approximations
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)
            # Compute absolute error
            num_iterations[j,i] = N
            absolut_error_hp_Gauss[j, i] = np.abs(np.abs(psi_nu_vec[2]) - np.abs(Q_psi_nu_vec[2]))


    # Plot results
    length = math.ceil(len(l_values) / 2)
    fig, axs = plt.subplots(2, length, figsize=(18.5, 8.9))

    if length == 1:
        axs = axs.flatten()

    plt.suptitle(r'Theoretical and Practical Effort for Node Points $hp$-Gauss', fontsize=15)

    for j, l in enumerate(l_values[:length]):
        axs[0, j].semilogy(num_iterations[j, :] ** (1/2), absolut_error_hp_Gauss[j, :], label=f"Practical l={l}")
        axs[0, j].semilogy(n_hp_Gauss_values[:, j] ** (1 / 2), Y_epsilon, marker='o', linestyle='-', label=f"Theoretical l={l}")
        axs[0, j].set_title(f"l = {l}")
        axs[0, j].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$', fontsize=8)
        axs[0, j].set_ylim(None, 10 ** 0 + 1)
    axs[0, 0].set_ylabel("Absolute Error")

    for j, l in enumerate(l_values[length:]):
        axs[1, j].semilogy(num_iterations[length+j, :] ** (1/2), absolut_error_hp_Gauss[length+j, :], label=f"Practical l={l}")
        axs[1, j].semilogy(n_hp_Gauss_values[:,length+ j] ** (1 / 2), Y_epsilon, marker='o', linestyle='-', label=f"Theoretical l={l}")
        axs[1, j].set_title(f"l = {l}")
        axs[1, j].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$', fontsize=8)
        axs[1, j].set_ylim(None, 10 ** 0 + 1)

    axs[1, 0].set_ylabel("Absolute Error")
    plt.subplots_adjust(hspace=0.4, wspace=0.3)
    legend = fig.legend(["Practical", "Theoretical"], loc="upper center", bbox_to_anchor=(0.5, 0.95), ncol=2,
                        fontsize=10)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

In [ ]:
def plot_comparison_effor_gauss_laguerre(n_values, l_values, exponent_max, C_list, alpha_list, C_tilde_mean, gamma, compute_psi_nu_quadrature):
    """
    Computes and plots the theoretical and practical effort for Gauss-Laguerre quadrature.

    Parameters:
    -----------
    n_values : array-like
        Number of quadrature points.
    l_values : array-like
        List of exponent values l.
    exponent_max : int
        Maximum exponent for the error tolerance.
    C_list : array-like
        List of C values for different l.
    alpha_list : array-like
        List of alpha values for different l.
    gamma : int
        Exponent for the n.
    compute_psi_nu_quadrature : function
        Function to compute psi_nu and quadrature values.
    """
    # Define epsilon function and theoretical node estimation
    epsilon = lambda x: 10 ** (-x)
    nodes_number_GLag = lambda rho, epsilon, C, alpha, gamma: (
        (2 / (1 + np.abs(rho))) *
        ((-1 / alpha) * np.log(epsilon / (C * 2.1 * (2 / (1 + np.abs(rho))) ** (nu + 1) * np.sqrt(2 * np.pi)))) ** gamma
    )

    # Initialize storage matrices
    n_GLag_values = np.zeros((exponent_max, len(l_values)))
    for i in range(0, exponent_max + 1):
        for j, l in enumerate(l_values):
            rho = 2.0 ** (-l)
            eps_value = epsilon(i)
            n_GLag_values[i - 1, j] = np.ceil(nodes_number_GLag(rho, eps_value, C_list[j], alpha_list[j], gamma))

    Y_epsilon = np.array([epsilon(x) for x in range(0, exponent_max)])
    absolut_error_GLag = np.zeros((len(l_values), len(n_values)))

    for i, n in enumerate(n_values):
        for j, l in enumerate(l_values):
            # Compute psi_nu and quadrature approximations
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)
            # Compute absolute error
            absolut_error_GLag[j, i] = np.abs(np.abs(psi_nu_vec[1]) - np.abs(Q_psi_nu_vec[1]))


    # Plot results
    length = math.ceil(len(l_values) / 2)
    fig, axs = plt.subplots(2, length, figsize=(18.5, 8.9))

    if length == 1:
        axs = axs.flatten()

    plt.suptitle(r'Theoretical and Practical Effort for Node Points Gauss-Laguerre', fontsize=15)

    for j, l in enumerate(l_values[:length]):
        axs[0, j].semilogy(n_values ** (1/2), absolut_error_GLag[j, :], label=f"Practical l={l}")
        axs[0, j].semilogy(n_GLag_values[:, j] ** (1 / 2), Y_epsilon, marker='o', linestyle='-', label=f"Theoretical l={l}")
        axs[0, j].set_title(f"l = {l}")
        axs[0, j].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$', fontsize=8)
        axs[0, j].set_ylim(None, 10 ** 0 + 1)
    axs[0, 0].set_ylabel("Absolute Error")

    for j, l in enumerate(l_values[length:]):
        axs[1, j].semilogy(n_values ** (1/2), absolut_error_GLag[length + j, :], label=f"Practical l={l}")
        axs[1, j].semilogy(n_GLag_values[:, length +  j] ** (1 / 2), Y_epsilon, marker='o', linestyle='-', label=f"Theoretical l={l}")
        axs[1, j].set_title(f"l = {l}")
        axs[1, j].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$', fontsize=8)
        axs[1, j].set_ylim(None, 10 ** 0 + 1)
    axs[1, 0].set_ylabel("Absolute Error")

    plt.subplots_adjust(hspace=0.4, wspace=0.3)
    legend = fig.legend(["Practical", "Theoretical"], loc="upper center", bbox_to_anchor=(0.5, 0.95), ncol=2,
                        fontsize=10)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

This function is computes the required number of quadrature nodes for different integration methods and plots $N(l)$ vs absolute error.

In [ ]:
def plot_nodes_vs_epsilon(l_values, exponent_max, kappa_list, C_list, b_list, C_tilde_mean, alpha_list, gamma=1.6):
    """
    Computes the required number of quadrature nodes for different integration methods and plots N(l) vs epsilon.

    Parameters:
    - l_values: List of l values
    - exponent_max: Maximum exponent for epsilon computation
    - kappa_list: List of kappa values corresponding to l_values
    - C_list: List of C values corresponding to l_values
    - C_tilde_mean: Mean value of C_tilde
    - alpha_list: List of alpha values corresponding to l_values
    - gamma: Default value for the gamma parameter (default=1.6)

    Returns:
    - A semi-logarithmic plot of N(l) vs absolutely error.
    """

    # Define the epsilon function
    epsilon = lambda x: 10 ** (-x)

    # Define functions for computing the number of nodes
    nodes_number_GLeg = lambda rho, epsilon, kappa: (1 / (2 * np.log(kappa))) * (
        np.log(9) - (2 * nu + 1) * np.log(np.abs(rho))
        - (2 * nu + 2) * np.log(1 - 0.5 * np.abs(rho))
        - np.log(epsilon)
    )

    nodes_number_hp_GLeg = lambda rho, epsilon, C, b: (
        (np.log(C) + (-2 * nu - 1) * np.log(1 - np.abs(rho))
        + (2 * nu + 2) * np.log(2 / (1 + np.abs(rho))) - np.log(epsilon)) / b
    ) ** 2

    nodes_number_GLag = lambda rho, epsilon, C, alpha, gamma: (
        (2 / (1 + np.abs(rho))) *
        ((-1 / alpha) * np.log(epsilon / (C * 2.1 * (2 / (1 + np.abs(rho))) ** (nu + 1) * np.sqrt(2 * np.pi)))) ** gamma
    )



    # Compute epsilon values
    Y_eps_potenz = np.arange(1, exponent_max + 1)
    Y_epsilon = np.array([epsilon(x) for x in range(1, exponent_max + 1)])

    # Initialize matrices for storing the number of nodes
    n_matrices = {name: np.zeros((exponent_max + 1, len(l_values) + 1))
                  for name in ["GLeg", "GLag", "hpGauss"]}

    # Set first row (l values) and first column (epsilon exponents)
    for name, matrix in n_matrices.items():
        matrix[0, 1:] = np.arange(1, len(l_values) + 1)  # First row: l values
        matrix[1:, 0] = Y_eps_potenz  # First column: epsilon exponents

    # Unpack matrices for direct use
    n_GLeg_values, n_GLag_values, n_hpGauss_values = n_matrices.values()

    # Compute nodes for each epsilon and l
    for i in range(1, exponent_max + 1):
        for j, l in enumerate(l_values):
            rho = 2.0 ** (-l)
            eps_value = epsilon(i)

            n_GLeg_values[i, j + 1] = np.ceil(nodes_number_GLeg(rho, eps_value, kappa_list[j]))
            n_hpGauss_values[i, j + 1] = np.ceil(nodes_number_hp_GLeg(rho, eps_value, C_list[j], b_list[j]))
            n_GLag_values[i, j + 1] = np.ceil(nodes_number_GLag(rho, eps_value, C_tilde_mean / 2, alpha_list[j], gamma))

    # Compute total nodes required
    n_result = n_GLeg_values + n_GLag_values + n_hpGauss_values

    # Determine min and max node values for plotting
    n_min = int(np.min(n_result[1:, 1:]))
    n_max = int(np.max(n_result[1:, 1:]))
    n_values = np.arange(n_min, n_max + 1)

    # Create figure
    fig, ax = plt.subplots(figsize=(10, 6))

    # Define colormaps
    winter_cmap = cm.get_cmap('winter', len(l_values))
    red_cmap = cm.get_cmap('RdPu', 5)

    # Plot N(l) vs epsilon for different l-values
    for i, l in enumerate(l_values, start=1):
        ax.semilogy(n_result[1:, i] ** (1/2), Y_epsilon, marker='o', linestyle='-',
                    label=f"$l={l}$", color=winter_cmap(i / len(l_values)))

    # Define reference curves
    ref_params = [
        (11/10, 1/2, 10.0 ** 1, exponent_max, r' 10 \, e^{ - \frac{11}{10} \, n^{1/2}}', red_cmap(2)),
        (3/4, 1/2, 10 ** 11, exponent_max, r'10^{11} \, e^{ - \frac{3}{4} \, n^{1/2}}', red_cmap(3))
    ]

    # Plot reference curves
    for factor, exponent, offset, upper_bound, label, color in ref_params:
        f_of_n = offset * np.exp(factor * -n_values ** exponent)
        valid_indices = np.where((f_of_n >= 10 ** (-exponent_max)) & (f_of_n <= 10 ** (-1)))[0]
        n_values_filtered = n_values[valid_indices]
        f_of_n_filtered = f_of_n[valid_indices]

        ax.plot(n_values_filtered ** (1/2), f_of_n_filtered,
                label=fr'ref: ${label}$',
                color=color)

    ax.set_title("Semilogy Plot: $N(l)$ vs Absolut Error")
    ax.set_xlabel("$N(l)^{1/2}$ (Total Number of Quadrature Nodes)")
    ax.set_ylabel("Absolut Error")
    ax.legend(loc="center left", bbox_to_anchor=(1.05, 0.5)) 
    ax.grid(True)

    plt.tight_layout()
    plt.show()

Define quadrature parameters

In [ ]:
n_values_GLeg = np.arange(2,45)
n_values_hpGLeg = np.arange(2,55)
n_values_GLag = np.arange(2,80)


l_values = np.arange(1,11)


exponent_max = 15

Calculation of Constants and Comparison

In [ ]:
print(f"------------GLeg-------------------")
kappa_list, kappa_mean = compute_kappa(n_values_GLeg, l_values)

kappa_list = replace_inf_with_mean(kappa_list)
kappa_list = [x + 0.2 for x in kappa_list]

plot_comparison_effort_gauss_legendre(n_values_GLeg, l_values, exponent_max, kappa_list, compute_psi_nu_quadrature)

print(f"------------hp-GLeg-------------------")
C_list, b_list, C_mean, b_mean = compute_C_b_hp_gauss(n_values_hpGLeg, l_values , 2)
C_list = [x for x in C_list]
b_list = [x for x in b_list]

plot_comparison_effort_hp_gauss(n_values_hpGLeg, l_values, exponent_max, C_list, b_list, compute_psi_nu_quadrature)



print(f"------------GLag-------------------") 

test = compute_constants_via_newton(n_values, l_values)
 if not np.isnan(test[2]):  # Falls kein NaN
     gamma = test[2]
else:
     print("Warning: gamma is NaN, using default value 1.4")
     gamma = 1.425


C_tilde_list, alpha_list, C_tilde_mean, alpha_mean = compute_C_alpha_glag(n_values_GLag, l_values, gamma)

plot_comparison_effor_gauss_laguerre(n_values_GLag, l_values, exponent_max, C_tilde_list, alpha_list, C_tilde_mean, gamma, compute_psi_nu_quadrature)


Plots the semilogarithmic relationship between the number of quadrature points and the required tolerance $\varepsilon = e^{-x}$.